In [1]:
import os
import pandas as pd

import boto3
import pyarrow.parquet as pymysql
from dotenv import load_dotenv
from io import BytesIO


In [6]:
all_users_media_prepro = pd.read_parquet("/home/flexmatch_ftp/ftp/multimodal_categorizer/all_users_media_prepro.parquet")
# all_users_media_prepro.info()

In [7]:
data = all_users_media_prepro[['member_uid', 'acnt_id', 'acnt_nm', 'acnt_sub_nm', 'intro_txt', 'media_id', 'spelled_checked_media_cn',
                                'main_category', 'top_3_category', 'image_path']].rename(columns={'spelled_checked_media_cn' : 'media_text'})

In [ ]:
load_dotenv('/home/flexmatch_ftp/ftp/multimodal_categorizer/config/.env')

aws_access_key = os.getenv("aws_accessKey")
aws_secret_key = os.getenv("aws_secretKey")
region_name='ap-northeast-2'

# S3 클라이언트 생성
s3_client = boto3.client('s3', aws_access_key_id=aws_access_key, aws_secret_access_key=aws_secret_key, region_name=region_name)

def list_parquet_files(bucket_name, prefix):
    """
    S3 버킷 내 지정된 prefix부터 시작하여 모든 .jpg 파일의 경로를 재귀적으로 탐색.
    """
    files = []
    paginator = s3_client.get_paginator('list_objects_v2')

    # 현재 prefix에서 객체와 하위 폴더 탐색
    # response = s3_client.list_objects_v2(Bucket=bucket_name, Prefix=prefix, Delimiter='/')

    # 현재 prefix에서 .jpg 파일 추가
    for response in paginator.paginate(Bucket=bucket_name, Prefix=prefix, Delimiter='/'):
        if 'Contents' in response:
            for content in response['Contents']:
                if content['Key'].endswith('.jpg'):
                    files.append(content['Key'])

        # 하위 폴더가 있으면 재귀적으로 탐색
        if 'CommonPrefixes' in response:
            for prefix_dir in response['CommonPrefixes']:
                new_prefix = prefix_dir['Prefix']
                files.extend(list_parquet_files(bucket_name, new_prefix))

    return files

In [45]:
# 폴더 구조와 파일 탐색
bucket_name = 'flexmatch-data'
prefix = 'instagram-data/tables/INSTAGRAM_POST_IMAGE_SAMPLE/media_image/'
parquet_files = list_parquet_files(bucket_name, prefix)

In [50]:
parquet_files[:1]

['instagram-data/tables/INSTAGRAM_POST_IMAGE_SAMPLE/media_image/17841400000175995/17915035293095890.jpg']

In [ ]:
img_path_df = pd.DataFrame(parquet_files, columns=['s3_path'])

In [66]:
img_path_df['acnt_id'] = img_path_df['s3_path'].str.split("/", expand=True)[4]
img_path_df['media_id'] = img_path_df['s3_path'].str.split("/", expand=True)[5]
img_path_df['media_id'] = img_path_df['media_id'].str.split(".", expand=True)[0]
# img_path_df['s3_path'].str.split("/", expand=True)[4]

In [ ]:
# img_path_df.to_parquet("/home/flexmatch_ftp/ftp/multimodal_categorizer/img_path_df.parquet")

In [8]:
all_data = pd.merge(data, img_path_df, on=['acnt_id', 'media_id'], how='inner')
all_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 132078 entries, 0 to 132077
Data columns (total 11 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   member_uid      132078 non-null  int64 
 1   acnt_id         132078 non-null  object
 2   acnt_nm         132078 non-null  object
 3   acnt_sub_nm     132078 non-null  object
 4   intro_txt       132078 non-null  object
 5   media_id        132078 non-null  object
 6   media_text      132078 non-null  object
 7   main_category   132078 non-null  object
 8   top_3_category  132078 non-null  object
 9   image_path      132078 non-null  object
 10  s3_path         132078 non-null  object
dtypes: int64(1), object(10)
memory usage: 11.1+ MB


In [ ]:
# all_data.to_parquet("/home/flexmatch_ftp/ftp/multimodal_categorizer/all_user_image_data.parquet")